# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [637]:
problem = '''The PRODA, S.A. industrial products firm has to face the problem of scheduling
the weekly production of its three products (P1, P2 and P3). These products are
sold to large industrial firms and PRODA, S.A. wishes to supply its products in
quantities that are more profitable for it.

Each product entails three operations contributing to the costs: smelting; mechanisation; assembly and
packaging. The smelting operations for products P1 and P2 could be subcontracted, but the smelting operation for product P3 requires special equipment, thus
preventing the use of subcontracts. PRODA also want to know, how much they should subcontract.

For product P1 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.30$
- subcontracted smelting: 0.50$
- mechanisation: 0.20$
- Assembly and packaging: 0.3$
The unit sales price is 1.50$.

For product P2 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.50$
- subcontracted smelting: 0.60$
- mechanisation: 0.10$
- Assembly and packaging: 0.20$
The unit sales price is 1.80$.

For product P3 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.40$
- mechanisation: 0.27$
- Assembly and packaging: 0.20$
The unit sales price is 1.97$.

Each unit of product P1 requires 6 min of smelting time (if performed at PRODA, S.A.), 6 min of mechanisation time and 3 min of assembly and packaging time, respectively. For product P2, the times are 10, 3 and 2 min, respectively. One unit of product P3 needs 8 min of smelting time, 8 min of mechanisation and 2 min for assembly and packaging. PRODA, S.A. has weekly capacities of 8,000 min of smelting time, 12,000 min of mechanisation time and 10,000 min of assembly and packaging time.
The objective is to maximize weekly profits.'''

## 2. Ask for parameters

In [691]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [692]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

**Production Quantities:**

*  *x*<sub>1</sub>: Number of units of product P1 produced and sold weekly.
*  *x*<sub>2</sub>: Number of units of product P2 produced and sold weekly.
*  *x*<sub>3</sub>: Number of units of product P3 produced and sold weekly.

**Subcontracting Quantities:**

*  *s*<sub>1</sub>: Number of units of product P1 subcontracted for smelting weekly.
*  *s*<sub>2</sub>: Number of units of product P2 subcontracted for smelting weekly. 


# 2. Ask for objective

In [710]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [711]:
Markdown(response2.text)

Objective Function (Maximize Profit):

```
Maximize Z = (1.50 - 0.30)x<sub>1</sub> - 0.20x<sub>1</sub> - 0.30x<sub>1</sub> + (1.80 - 0.50)x<sub>2</sub> - 0.10x<sub>2</sub> - 0.20x<sub>2</sub> + (1.97 - 0.40)x<sub>3</sub> - 0.27x<sub>3</sub> - 0.20x<sub>3</sub> - 0.20s<sub>1</sub> - 0.10s<sub>2</sub> 
```

Simplifying:

```
Maximize Z = 0.70x<sub>1</sub> + 1.00x<sub>2</sub> + 1.07x<sub>3</sub> - 0.20s<sub>1</sub> - 0.10s<sub>2</sub>
```

This objective function calculates the total profit by considering the following for each product:

1.  **Revenue:** Selling price per unit multiplied by the number of units sold.
2.  **Production Costs:** Costs of smelting, mechanization, and assembly/packaging at PRODA.
3.  **Subcontracting Costs:**  Costs associated with subcontracting smelting for P1 and P2. 


# 3. Ask for constraints

In [712]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [713]:
Markdown(response3.text)

## Constraints:

**1. Production Capacity Constraints:**

*   **Smelting:** 6(*x*<sub>1</sub> - *s*<sub>1</sub>) + 10(*x*<sub>2</sub> - *s*<sub>2</sub>) + 8*x*<sub>3</sub> ≤ 8000  (minutes)
*   **Mechanization:** 6*x*<sub>1</sub> + 3*x*<sub>2</sub> + 8*x*<sub>3</sub> ≤ 12000 (minutes)
*   **Assembly and Packaging:** 3*x*<sub>1</sub> + 2*x*<sub>2</sub> + 2*x*<sub>3</sub> ≤ 10000 (minutes)

**2. Subcontracting Limits:**

*   0 ≤ *s*<sub>1</sub> ≤ *x*<sub>1</sub>  (Can't subcontract more than what's produced of P1)
*   0 ≤ *s*<sub>2</sub> ≤ *x*<sub>2</sub> (Can't subcontract more than what's produced of P2)

**3. Non-Negativity Constraints:**

*   *x*<sub>1</sub>, *x*<sub>2</sub>, *x*<sub>3</sub>, *s*<sub>1</sub>, *s*<sub>2</sub> ≥ 0 


## 3. Generate the pyomo code

In [714]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [715]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P1 production
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P2 production
model.x3 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P3 production
model.s1 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P1 subcontracted smelting
model.s2 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P2 subcontracted smelting

# Define the objective function
model.profit = pyo.Objective(
    expr=0.70 * model.x1 + 1.00 * model.x2 + 1.07 * model.x3 - 0.20 * model.s1 - 0.10 * model.s2,
    sense=pyo.maximize
)

# Define constraints
model.smelting_constraint = pyo.Constraint(
    expr=6 * (model.x1 - model.s1) + 10 * (model.x2 - model.s2) + 8 * model.x3 <= 8000
)
model.mechanization_constraint = pyo.Constraint(
    expr=6 * model.x1 + 3 * model.x2 + 8 * model.x3 <= 12000
)
model.assembly_constraint = pyo.Constraint(
    expr=3 * model.x1 + 2 * model.x2 + 2 * model.x3 <= 10000
)
model.subcontracting_p1_constraint = pyo.Constraint(
    expr=0 <= model.s1 <= model.x1
)
model.subcontracting_p2_constraint = pyo.Constraint(
    expr=0 <= model.s2 <= model.x2
)

# Solve the model
# You can choose a solver that you have installed, for example:
# solver = pyo.SolverFactory('glpk') 
# results = solver.solve(model)

# Print the results
# print(results)
# print(f"Optimal Profit: {pyo.value(model.profit)}")
# print(f"Product P1 Production: {pyo.value(model.x1)}")
# print(f"Product P2 Production: {pyo.value(model.x2)}")
# print(f"Product P3 Production: {pyo.value(model.x3)}")
# print(f"Product P1 Subcontracted Smelting: {pyo.value(model.s1)}")
# print(f"Product P2 Subcontracted Smelting: {pyo.value(model.s2)}") 
``` 


## 4. Input problem data and try running the generated code

In [717]:
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P1 production
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P2 production
model.x3 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P3 production
model.s1 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P1 subcontracted smelting
model.s2 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P2 subcontracted smelting

# Define the objective function
model.profit = pyo.Objective(
    expr=0.70 * model.x1 + 1.00 * model.x2 + 1.07 * model.x3 - 0.20 * model.s1 - 0.10 * model.s2,
    sense=pyo.maximize
)

# Define constraints
model.smelting_constraint = pyo.Constraint(
    expr=6 * (model.x1 - model.s1) + 10 * (model.x2 - model.s2) + 8 * model.x3 <= 8000
)
model.mechanization_constraint = pyo.Constraint(
    expr=6 * model.x1 + 3 * model.x2 + 8 * model.x3 <= 12000
)
model.assembly_constraint = pyo.Constraint(
    expr=3 * model.x1 + 2 * model.x2 + 2 * model.x3 <= 10000
)
model.subcontracting_p1_constraint = pyo.Constraint(
    expr=0 <= model.s1 <= model.x1
)
model.subcontracting_p2_constraint = pyo.Constraint(
    expr=0 <= model.s2 <= model.x2
)

# Solve the model
# You can choose a solver that you have installed, for example:
solver = pyo.SolverFactory('glpk') 
results = solver.solve(model)

# Print the results
print(results)
print(f"Optimal Profit: {pyo.value(model.profit)}")
print(f"Product P1 Production: {pyo.value(model.x1)}")
print(f"Product P2 Production: {pyo.value(model.x2)}")
print(f"Product P3 Production: {pyo.value(model.x3)}")
print(f"Product P1 Subcontracted Smelting: {pyo.value(model.s1)}")
print(f"Product P2 Subcontracted Smelting: {pyo.value(model.s2)}") 

PyomoException: Cannot convert non-constant Pyomo expression (0  <=  s1) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

## 5. Correct the code to verify model viability (optional)

In [718]:
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P1 production
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P2 production
model.x3 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P3 production
model.s1 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P1 subcontracted smelting
model.s2 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P2 subcontracted smelting

# Define the objective function
model.profit = pyo.Objective(
    expr=0.70 * model.x1 + 1.00 * model.x2 + 1.07 * model.x3 - 0.20 * model.s1 - 0.10 * model.s2,
    sense=pyo.maximize
)

# Define constraints
model.smelting_constraint = pyo.Constraint(
    expr=6 * (model.x1 - model.s1) + 10 * (model.x2 - model.s2) + 8 * model.x3 <= 8000
)
model.mechanization_constraint = pyo.Constraint(
    expr=6 * model.x1 + 3 * model.x2 + 8 * model.x3 <= 12000
)
model.assembly_constraint = pyo.Constraint(
    expr=3 * model.x1 + 2 * model.x2 + 2 * model.x3 <= 10000
)
model.subcontracting_p1_constraint1 = pyo.Constraint(
    expr=0 <= model.s1 
)
model.subcontracting_p1_constraint = pyo.Constraint(
    expr= model.s1 <= model.x1
)
model.subcontracting_p2_constraint1 = pyo.Constraint(
    expr=0 <= model.s2
)
model.subcontracting_p2_constraint = pyo.Constraint(
    expr=model.s2 <= model.x2
)

# Solve the model
# You can choose a solver that you have installed, for example:
solver = pyo.SolverFactory('glpk') 
results = solver.solve(model)

# Print the results
print(results)
print(f"Optimal Profit: {pyo.value(model.profit)}")
print(f"Product P1 Production: {pyo.value(model.x1)}")
print(f"Product P2 Production: {pyo.value(model.x2)}")
print(f"Product P3 Production: {pyo.value(model.x3)}")
print(f"Product P1 Subcontracted Smelting: {pyo.value(model.s1)}")
print(f"Product P2 Subcontracted Smelting: {pyo.value(model.s2)}") 


Problem: 
- Name: unknown
  Lower bound: 3680.0
  Upper bound: 3680.0
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 5
  Number of nonzeros: 17
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04004049301147461
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal Profit: 3680.0
Product P1 Production: 0.0
Product P2 Production: 4000.0
Product P3 Production: 0.0
Product P1 Subcontracted Smelting: 0.0
Product P2 Subcontracted Smelting: 3200.0


## 6. Print the responses

In [723]:
print(response.text)

## Variables:

**Production Quantities:**

*  *x*<sub>1</sub>: Number of units of product P1 produced and sold weekly.
*  *x*<sub>2</sub>: Number of units of product P2 produced and sold weekly.
*  *x*<sub>3</sub>: Number of units of product P3 produced and sold weekly.

**Subcontracting Quantities:**

*  *s*<sub>1</sub>: Number of units of product P1 subcontracted for smelting weekly.
*  *s*<sub>2</sub>: Number of units of product P2 subcontracted for smelting weekly. 



In [724]:
print(response2.text)

Objective Function (Maximize Profit):

```
Maximize Z = (1.50 - 0.30)x<sub>1</sub> - 0.20x<sub>1</sub> - 0.30x<sub>1</sub> + (1.80 - 0.50)x<sub>2</sub> - 0.10x<sub>2</sub> - 0.20x<sub>2</sub> + (1.97 - 0.40)x<sub>3</sub> - 0.27x<sub>3</sub> - 0.20x<sub>3</sub> - 0.20s<sub>1</sub> - 0.10s<sub>2</sub> 
```

Simplifying:

```
Maximize Z = 0.70x<sub>1</sub> + 1.00x<sub>2</sub> + 1.07x<sub>3</sub> - 0.20s<sub>1</sub> - 0.10s<sub>2</sub>
```

This objective function calculates the total profit by considering the following for each product:

1.  **Revenue:** Selling price per unit multiplied by the number of units sold.
2.  **Production Costs:** Costs of smelting, mechanization, and assembly/packaging at PRODA.
3.  **Subcontracting Costs:**  Costs associated with subcontracting smelting for P1 and P2. 



In [725]:
print(response3.text)

## Constraints:

**1. Production Capacity Constraints:**

*   **Smelting:** 6(*x*<sub>1</sub> - *s*<sub>1</sub>) + 10(*x*<sub>2</sub> - *s*<sub>2</sub>) + 8*x*<sub>3</sub> ≤ 8000  (minutes)
*   **Mechanization:** 6*x*<sub>1</sub> + 3*x*<sub>2</sub> + 8*x*<sub>3</sub> ≤ 12000 (minutes)
*   **Assembly and Packaging:** 3*x*<sub>1</sub> + 2*x*<sub>2</sub> + 2*x*<sub>3</sub> ≤ 10000 (minutes)

**2. Subcontracting Limits:**

*   0 ≤ *s*<sub>1</sub> ≤ *x*<sub>1</sub>  (Can't subcontract more than what's produced of P1)
*   0 ≤ *s*<sub>2</sub> ≤ *x*<sub>2</sub> (Can't subcontract more than what's produced of P2)

**3. Non-Negativity Constraints:**

*   *x*<sub>1</sub>, *x*<sub>2</sub>, *x*<sub>3</sub>, *s*<sub>1</sub>, *s*<sub>2</sub> ≥ 0 



In [726]:
print(response4.text)

```python
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P1 production
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P2 production
model.x3 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P3 production
model.s1 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P1 subcontracted smelting
model.s2 = pyo.Var(domain=pyo.NonNegativeReals)  # Product P2 subcontracted smelting

# Define the objective function
model.profit = pyo.Objective(
    expr=0.70 * model.x1 + 1.00 * model.x2 + 1.07 * model.x3 - 0.20 * model.s1 - 0.10 * model.s2,
    sense=pyo.maximize
)

# Define constraints
model.smelting_constraint = pyo.Constraint(
    expr=6 * (model.x1 - model.s1) + 10 * (model.x2 - model.s2) + 8 * model.x3 <= 8000
)
model.mechanization_constraint = pyo.Constraint(
    expr=6 * model.x1 + 3 * model.x2 + 8 * model.x3 <= 12000
)
model.assembly_constraint = pyo.Constraint(
 